In [ ]:
import csv, sys, os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.metrics import accuracy_score
from sklearn import model_selection
#from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
#from sklearn.svm import SVC

def csv2lst(csv_Path):
    with open(csv_Path, 'r') as output:
        reader = csv.reader(output, lineterminator = '\n')
        lst = list(reader)
    return lst

In [ ]:
# Load dfs
colHeads = ['indx','avg_down_month','avg_drawdown','avg_drawdown_days','avg_up_month','best_day','best_month','cagr','calmar','daily_kurt','daily_mean','daily_sharpe','daily_skew','daily_sortino','daily_vol','max_drawdown','monthly_kurt','monthly_mean','monthly_sharpe','monthly_skew','monthly_sortino','monthly_vol','mtd','six_month','three_month','ticker_id','total_return','worst_day','worst_month','YTD','nextYTD','40p_increase']
print(len(colHeads))

df_Trn = pd.read_csv('secs - train.csv', names=colHeads)
df_Tst = pd.read_csv('secs - test.csv', names=colHeads)
testDataLst = csv2lst('secs - test.csv')

dfTrnIndx = pd.factorize(df_Trn['indx'])
dfTstIndx = pd.factorize(df_Tst['indx'])

In [ ]:
models = []
models.append(['LR', LogisticRegression(), [] ])
models.append(['LDA', LinearDiscriminantAnalysis(), [] ])
models.append(['KNN', KNeighborsClassifier(), [] ])
models.append(['CART', DecisionTreeClassifier(), [] ])
models.append(['NB', GaussianNB(), [] ])
#models.append(['SVM', SVC(), [] ])
#models.append(['RFC', RFC(n_jobs=2,n_estimators=50), [] ])

In [ ]:
arrayTrn = df_Trn.values # make np arrays
arrayTst = df_Tst.values 

trainData = arrayTrn[:,1:31] #values
trainData = trainData.astype('float')

trainAnswers = arrayTrn[:,31] #answer
trainAnswers = trainAnswers.astype('float')

testData = arrayTst[:,1:31]
testData = testData.astype('float')

testAnswers = arrayTst[:,31]
testAnswers = testAnswers.astype('float')

testIndex = arrayTst[:,0]


In [ ]:
cnt = 0
for name, model, predictions in models:
    model.fit(trainData, trainAnswers)
    predictions = model.predict(testData)
    
    score = accuracy_score(testAnswers, predictions)
    kfold = model_selection.KFold(n_splits=10, random_state=7)
    cv_results = model.cross_val_score(model, X_train, Y_train, cv=kfold, scoring=scoring)
    
    models[cnt][2] = predictions #store predictions back into models list
    models[cnt].append(score)
    print(name, score, cv_results.mean(), cv_results.std())
    cnt += 1

In [ ]:
writeOut = []
for i in range(len(testDataLst)):
    writeOut.append([testDataLst[i], 
                     testIndex[i], 
                     testAnswers[i], 
                     models[0][2][i], 
                     models[1][2][i], 
                     models[2][2][i], 
                     models[3][2][i], 
                     models[4][2][i], 
                     models[5][2][i], 
                     models[5][2][i] ])

df2csv = pd.DataFrame(writeOut, columns=['testData','testIndex','testAnswers','LR_pred','LDA_pred','KNN_pred','CART_pred','NB_pred','SVM_pred','RFC_pred'])
df2csv.to_csv('pred.csv', index=False)